## Segmenting and Clustering Neighborhoods in Toronto - 2nd Part

This is the notebook that contains the code for the seconde part of the week 3 assignment of the course Applied Data Science Capstone. 

In order to parse the HTML page, I decided to use __*BeautifulSoup*__

In [3]:
#!conda install -c anaconda beautifulsoup4 --yes 

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
# Read the URL page at Wikipedia
data = BeautifulSoup(page.text, "html.parser")

# Get the HTML table inside the HTML data read
table = data.find('table')

# Get all the rows of the HTML table
table_rows = table.find_all('tr')

# Get the header TH
header = table_rows[0].find_all('th')

# Get the column values (TD) of the header TH, which will be the column labels of the dataframe
labels = [table_rows[0].text for table_rows[0] in header]
# Clean the '\n' special character
labels = np.char.replace(labels, '\n', '')

# Create the dataframe
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(columns = labels)

# Get all the other columns (TD) from the rows (TR)
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    # Ignore the first row (header) and the rows which have "Not assigned" boroughs
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
    if (row != [] and row[1] != "Not assigned\n"):
        row = pd.Series(row,labels)
        df = df.append([row],ignore_index=True)     

# Clean the data removing '\n'        
df.replace('\n','', regex=True, inplace=True)   

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
arr = np.where(pd.isnull(df['Neighborhood']))
print(arr)
# there are no 'not assigned' neighborhoods in the dataframe



In [ ]:
# Read the geospatial data csv
geospatial = pd.read_csv("http://cocl.us/Geospatial_data") 
geospatial.head()

In [ ]:
# Change the dataframes indexes, in order to make it possible to merge them later
df.set_index('Postal Code', inplace=True)
geospatial.set_index('Postal Code', inplace=True)

In [ ]:
# Merge both dataframes
mergedDf = df.merge(geospatial, left_index=True, right_index=True)
mergedDf

In [ ]:
# Transform Postal Code into a column again
mergedDf['CEP'] = mergedDf.index
mergedDf.reset_index(inplace=True)
del mergedDf['Postal Code']
mergedDf.rename(columns={'CEP':'Postal Code'}, inplace=True)

mergedDf.head()

In [ ]:
# Define Postal Code as the first column
cols = mergedDf.columns.tolist()
cols = cols[-1:] + cols[:-1]
mergedDf = mergedDf[cols]
mergedDf.head()

In [ ]:
mergedDf
